In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop

tfd = tfp.distributions
tfpl = tfp.layers

2021-07-15 10:26:27.476099: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.2


In [13]:
tf.__version__

'2.5.0'

In [14]:
tfp.__version__


'0.13.0'

In [2]:
def nll(y_true, y_pred):
    return -y_pred.log_prob(y_true)

In [4]:
def create_probabilistic_model(input_shape, loss, optimizer, metrics):
    model = Sequential([
        Conv2D(8, 5, activation='relu', padding='valid', input_shape=input_shape),
        MaxPooling2D(6),
        Flatten(),
        Dense(10),
        tfpl.OneHotCategorical(10)
    ])
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [5]:
probabilistic_model = create_probabilistic_model(input_shape=(28, 28, 1), loss=nll,optimizer=RMSprop(),metrics=['accuracy'])

2021-07-15 10:29:07.581250: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-15 10:29:07.664000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-15 10:29:07.665458: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA TITAN Xp computeCapability: 6.1
coreClock: 1.582GHz coreCount: 30 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 510.07GiB/s
2021-07-15 10:29:07.665519: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-15 10:29:07.679762: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-15 10:29:07.679901: I tensorflow/stream_executor/platform/d

In [6]:
probabilistic_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 8)         208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
one_hot_categorical (OneHotC multiple                  0         
Total params: 1,498
Trainable params: 1,498
Non-trainable params: 0
_________________________________________________________________


In [8]:
## Save method 1, save in h5 format
probabilistic_model.save('/tmp/probabilistic_model.h5')

In [9]:
# load from method 1 (Error while using h5 format while loading the model is shown below.)
loaded_model = tf.keras.models.load_model('/tmp/model/probabilistic_model.h5')

OSError: SavedModel file does not exist at: /tmp/model/probabilistic_model.h5/{saved_model.pbtxt|saved_model.pb}

In [10]:
# Save method 2 (Workaround works with limited capability as shown in https://github.com/tensorflow/probability/issues/325#issuecomment-477213850)
import h5py
#probabilistic_model.save('/tmp/model/probabilistic_model') #(error)
model_name='probabilistic_model'
file = h5py.File('/tmp/{}.h5'.format(model_name), 'w')
weight = probabilistic_model.get_weights()
for i in range(len(weight)):
   file.create_dataset('weight' + str(i), data=weight[i])
file.close()

In [11]:
# load from method 2 (Workaround works with limited capability as shown in https://github.com/tensorflow/probability/issues/325#issuecomment-477213850)

file = h5py.File('/tmp/{}.h5'.format(model_name), 'r')
weight = []
for i in range(len(file.keys())):
   weight.append(file['weight' + str(i)][:])
probabilistic_model.set_weights(weight)


In [12]:
# Save method 3
# expected behavior
# Saves a probabilistic_model as a TensorFlow SavedModel 
# as per "https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model"
probabilistic_model.save('/tmp/model/probabilistic_model')

OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.